In [3]:
 %load_ext autoreload


# From ZK to Bulletproofs

In [4]:
from klefki.types.algebra.concrete import EllipticCurveCyclicSubgroupSecp256k1 as Cruve
from klefki.types.algebra.concrete import EllipticCurveGroupSecp256k1 as ECG
from klefki.types.algebra.concrete import FiniteFieldCyclicSecp256k1 as CF

G = Cruve.G

## 2.3 A Pedersen commitment in elliptic curve form

In [5]:
import random

N = 0xfffffffffffffffffffffffffffffffebaaedce6af48a03bbfd25e8cd0364141
def random_cf() -> CF: return CF(random.randint(1, N))

$$C=rH+aG$$

$H$ is a curve point, for which nobody knows the discrete logarithm $q$ s.t. $H = qG$.

In [6]:
q = random_cf()
H = G@q

In [7]:
def C(r: CF, a: CF): return (H@r + G@a)

### Homomorphism

In [8]:
r_1, a_1, r_2, a_2 = random_cf(), random_cf(), random_cf(), random_cf()

In [9]:
assert C(r_1, a_1) + C(r_2, a_2) == H@r_1 + G@a_1 + H@r_2 + G@a_2

In [10]:
assert C(r_1, a_1) + C(r_2, a_2) == C(r_1 + r_2, a_1 + a_2)

## 2.4 The Vector Pedersen Commitment

To extend to a more powerful form of the Pedersen commitment already deﬁned, we go from:
    $$C = rH + aG$$
    to:
        $$C=rH + \sum_{i=0}^n v_iG_i = rH + \mathbf{v}\mathbf{G}$$

The individual G i s can be constructed using a simple algorithm of the form already mentioned (like, take a $\mathcal{H}(encode(G)||i)$ where $\mathcal{H}$ represents some hash function).

In [40]:
from klefki.types.algebra.utils import encode, decode
from hashlib import sha256

In [41]:
G_is = list(
    map(decode,
        (map(lambda x: sha256(x.encode()).hexdigest(), encode(G)))
       )
)

## 3 A zero knowledge argument of knowledge of a set of vectors

### Schnorr’s identity protocol

Prover starts with a public key P and a corresponding private key $x$ s.t. $P = xG$.
Prover wishes to prove in zero knowledge, that he knows $x$.

1. $P\rightarrow V:xG$
2. $P\rightarrow V:R$ (a new random curve point, but $P$ knows $k$ s.t. $R=kG$)
3. $V\rightarrow P:e$ (a random scalar)
4. $P\rightarrow V:s$ (which $P$ calcuated from the quation $s=k+ex$)

In [51]:
P, V = {}, {}
# step 1
P['x'] = random_cf()
V['P'] = G @ P['x']

# step 2
P['k'] = random_cf()
V['R'] = G @ P['k']

# step 3
V['e'] = random_cf()
P['e'] = V['e']

# step 4
P['s'] = P['k'] + P['e'] * P['x']
V['s'] = P['s']



Veriﬁcation works fairly trivially: veriﬁer checks $sG =?R+eP$.

The “Simulator”, which controls the execution of the veriﬁer, given the public key P, just as the Veriﬁer would be, can fake a valid transcript as follows:

Choose $s$ randomly. Then, choose $e$, also randomly. Finally, we only need to choose $R$ to create a complete conversation transcript; it must be $R = sG − eP$. 

$$
R=sG-eP\\
kG=sG-eP\\
sG=kG+eP\\
sG=kG+xeG\\
s=k+ex\\
$$

Then we have successfully simulated a conversation which is entirely valid: $(R, e, s)$, without ever knowing the secret key $x$, and which it’s easy to see is randomly distributed in the same way as a real one would be ($R$ is a free variable).

In [53]:
s, e = random_cf(), random_cf()
R = G @ V['s'] - V['P'] @ V['e']

In [56]:
assert R == V['R']

### vector proof of knowledge